In [8]:
import pandas
import openpyxl #leer y escribir en excel 
import sqlalchemy, pymysql #conectar python con la base de datos Mysql

# ETL

In [24]:
import pandas as pd
import numpy as np
import os 
import glob

In [10]:
RUTA_TR = "./DCE_DATOS_2025/TR"

MAPA_SENSORES = {
    "01_VOLTAJE_AC_DEL_SISTEMA_L1_L2": "VOLTAJE_AC_L1_l2",
    "02_VOLTAJE_AC_DEL_SISTEMA_L2-L3": "VOLTAJE_AC_L2_l3",
    "03_VOLTAJE_AC_DEL_SISTEMA_L3-L1": "VOLTAJE_AC_L3_l1",
    "11_CORRIENTE_AC_DE_LA_CARGA_L1": "CORRIENTE_AC_L1",
    "12_CORRIENTE_AC_DE_LA_CARGA_L2": "CORRIENTE_AC_L2",
    "13_CORRIENTE_AC_DE_LA_CARGA_L3": "CORRIENTE_AC_L3",
    "14_POTENCIA_ACTIVA_DE_LA_CARGA": "POTENCIA_ACTIVA_KW",
    "15_POTENCIA_REACTIVA_DE_LA_CARGA": "POTENCIA_REACTIVA_KVAR",
    "16_POTENCIA_APARENTE_DE_LA_CARGA": "POTENCIA_APARENTE_KVA",
    "17_FACTOR_DE_POTENCIA_DE_LA_CARGA": "FACTOR_DE_POTENCIA"
}


In [21]:
def limpiar_fecha(fecha_str):
    """
    TR - IDEO CALI,"01 - VOLTAJE AC DEL SISTEMA L1-L2","216.0","V","1/1/2025, 12:50:03 a.Â m."

    Arregla el formato y elimina caracteres de codificación corruptos (Â).
    Entrada: '1/1/2025, 12:00:00 a.Â m.' o '1/1/2025, 12:00:00 a.\xa0m.'
    Salida: '1/1/2025 12:00:00 AM'
    """

    if not isinstance(fecha_str, str):
        return str(fecha_str)
    

    limpia = fecha_str.replace("Â", "") \
                      .replace("\xa0", " ") \
                      .replace(",", "") 
    
    limpia = limpia.lower() \
                   .replace("a. m.", "AM") \
                   .replace("p. m.", "PM") \
                   .replace("a.m.", "AM") \
                   .replace("p.m.", "PM") \
                   .replace("am", "AM") \
                   .replace("pm", "PM")
    
    return limpia.strip() # Eliminar espacios 

In [23]:
fecha= "12:50:03 a.Â m."
fecha1 = limpiar_fecha(fecha)
print(fecha1)

12:50:03 AM


In [26]:
def procesar_carpeta_sensor(nombre_carpeta, nombre_columna_db):
    """
    Lee los 12 meses de una carpeta, limpia y consolida.
    """

    ruta_busqueda = os.path.join(RUTA_TR, nombre_carpeta, "*.csv") # O *.cvs si tienen error de extension
    archivos = glob.glob(ruta_busqueda)
    
    if not archivos:
        print(f"ADVERTENCIA: No se encontraron archivos en {nombre_carpeta}")
        return None

    print(f"Procesando {nombre_carpeta} ({len(archivos)} archivos)...")
    
    dfs_meses = []
    
    for archivo in archivos:
        try:
            # Leemos el CSV. Importante: dtype=str para no romper la lectura con 'Unplugged'
            df_temp = pd.read_csv(archivo, encoding='latin-1', dtype=str) # Probamos latin-1 por los caracteres raros
            
            df_temp['Time_Clean'] = df_temp['Time'].apply(limpiar_fecha)
            df_temp['timestamp'] = pd.to_datetime(df_temp['Time_Clean'], format='%d/%m/%Y %I:%M:%S %p', errors='coerce')
            
            # Convertimos Value a números. Los textos se vuelven NaN.
            df_temp['valor_numerico'] = pd.to_numeric(df_temp['Value'], errors='coerce')
            
            #Eliminar filas con NaN
            df_temp = df_temp.dropna(subset=['valor_numerico', 'timestamp'])
            
            # Seleccionamos solo lo que sirve (ya limpio)
            df_util = df_temp[['timestamp', 'valor_numerico']].copy()
            
            dfs_meses.append(df_util)
            
        except Exception as e:
            print(f"Error leyendo {os.path.basename(archivo)}: {e}")

    if not dfs_meses:
        return None

    # Unimos Enero-Diciembre verticalmente
    df_anual = pd.concat(dfs_meses, ignore_index=True)
    
    # Ordenamos por fecha
    df_anual = df_anual.sort_values('timestamp')
    df_anual = df_anual.set_index('timestamp')
    
    # 3. RESAMPLING (Normalización de tiempo)
    # Como los datos vienen a 12:01:59 y otros a 12:05:00, promediamos cada 10 minutos
    # Esto alinea los datos y reduce el ruido.
    df_resampled = df_anual.resample('10min').mean()
    
    # Renombramos la columna 'valor_numerico' al nombre real del sensor (ej: voltaje_ac_l1_l2)
    df_resampled.columns = [nombre_columna_db]
    
    return df_resampled

In [27]:
if __name__ == "__main__":
    print("ETL para Elemento TR...")
    
    dataframes_sensores = []
    
    # Recorremos cada carpeta definida en el mapa
    for carpeta_original, columna_destino in MAPA_SENSORES.items():
        df_sensor = procesar_carpeta_sensor(carpeta_original, columna_destino)
        
        if df_sensor is not None:
            dataframes_sensores.append(df_sensor)
    
    if dataframes_sensores:
        print("\n Unificando todos los sensores en una sola tabla maestra...")
        
        # Unimos todos los dataframes usando el índice (timestamp)
        # axis=1 significa pegar las columnas una al lado de la otra
        df_final_tr = pd.concat(dataframes_sensores, axis=1)
        
        # Opcional: Llenar huecos pequeños (si un sensor falló 10 min) o dejarlos como NaN
        # df_final_tr = df_final_tr.interpolate(method='time') 
        
        # Reset index para que timestamp sea una columna
        df_final_tr = df_final_tr.reset_index()
        
        print(f" Tabla consolidada creada: {df_final_tr.shape[0]} registros x {df_final_tr.shape[1]} columnas")
        print(df_final_tr.head())
        
        # GUARDAR 
        archivo_salida = "consolidado_TR_2025.csv" 
        df_final_tr.to_csv(archivo_salida, index=False)
        print(f"\n Archivo guardado: {archivo_salida}")
        
    else:
        print("No se pudieron procesar datos.")

ETL para Elemento TR...
Procesando 01_VOLTAJE_AC_DEL_SISTEMA_L1_L2 (12 archivos)...
Procesando 02_VOLTAJE_AC_DEL_SISTEMA_L2-L3 (12 archivos)...
Procesando 03_VOLTAJE_AC_DEL_SISTEMA_L3-L1 (12 archivos)...
Procesando 11_CORRIENTE_AC_DE_LA_CARGA_L1 (12 archivos)...
Procesando 12_CORRIENTE_AC_DE_LA_CARGA_L2 (12 archivos)...
Procesando 13_CORRIENTE_AC_DE_LA_CARGA_L3 (12 archivos)...
Procesando 14_POTENCIA_ACTIVA_DE_LA_CARGA (12 archivos)...
Procesando 15_POTENCIA_REACTIVA_DE_LA_CARGA (12 archivos)...
Procesando 16_POTENCIA_APARENTE_DE_LA_CARGA (12 archivos)...
Procesando 17_FACTOR_DE_POTENCIA_DE_LA_CARGA (12 archivos)...

 Unificando todos los sensores en una sola tabla maestra...
 Tabla consolidada creada: 52560 registros x 11 columnas
            timestamp  VOLTAJE_AC_L1_l2  VOLTAJE_AC_L2_l3  VOLTAJE_AC_L3_l1  \
0 2025-01-01 00:00:00        215.250000        215.000000        216.750000   
1 2025-01-01 00:10:00        215.333333        215.333333        216.333333   
2 2025-01-01 00:20:00